## Working with Transactions and the DBAPI

With the `Engine` object ready to go, we may now proceed to dive into the basic operation of an `Engine` and its primary interactive endpoints, the `Connection` and `Result`. We will additionally introduce the `ORM`'s facade for these objects, known as the `Session`.

> When using the ORM, the `Engine` is managed by another object called the `Session`. The `Session` in modern SQLAlchemy emphasizes a transactional and SQL execution pattern that is largely identical to that of the `Connection` discussed below, so while this subsection is Core-centric, all of the concepts here are essentially relevant to ORM use as well and is recommended for all ORM learners. The execution pattern used by the `Connection` will be contrasted with that of the `Session` at the end of this section.

#### Getting a Connection

The sole purpose of the `Engine` object from a user-facing perspective is to provide a unit of connectivity to the database called the `Connection`. When working with the Core directly, the `Connection` object is how all interaction with the database is done. As the `Connection` represents an open resource against the database, we want to always limit the scope of our use of this object to a specific context, and the best way to do that is by using `Python context manager` form, also known as the `with statement`.

In [1]:
from sqlalchemy import create_engine, text

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
with engine.connect() as conn:
    result = conn.execute(text("select 'Hello World!'"))
    print(result.all())

2022-09-19 06:22:16,403 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-19 06:22:16,405 INFO sqlalchemy.engine.Engine select 'Hello World!'
2022-09-19 06:22:16,406 INFO sqlalchemy.engine.Engine [generated in 0.00284s] ()
[('Hello World!',)]
2022-09-19 06:22:16,408 INFO sqlalchemy.engine.Engine ROLLBACK


In this example, the context manager provided for a database connection and also framed the operation inside of a transaction. The default behavior of the `Python DBAPI` includes that **a transaction is always in progress**; when the scope of the connection is released, a __ROLLBACK__ is emitted to end the transaction. The transaction is _not committed_ automatically; when we want to commit data we normally need to call `Connection.commit()`.

> `autocommit` mode is available for special cases.

The result of our `SELECT` was also returned in an object called `Result` that will be discussed later, however for the moment we'll add that it's best to ensure this object is consumed within the `connect` block, and is not passed along outside of the scope of our connection.

#### Committing Changes

As learnt already that the `DBAPI connection is non-autocommitting`, what if we want to commit some data? We can create a table and insert some data, and the transaction is then committed using the `Connection.commit()` method, invoked inside the block where we acquired the `Connection` object.

In [3]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE table_name (x int, y int)"))
    conn.execute(
        text("INSERT INTO table_name (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2022-09-19 06:22:16,498 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-19 06:22:16,499 INFO sqlalchemy.engine.Engine CREATE TABLE table_name (x int, y int)
2022-09-19 06:22:16,501 INFO sqlalchemy.engine.Engine [generated in 0.00279s] ()
2022-09-19 06:22:16,504 INFO sqlalchemy.engine.Engine INSERT INTO table_name (x, y) VALUES (?, ?)
2022-09-19 06:22:16,506 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ((1, 1), (2, 4))
2022-09-19 06:22:16,507 INFO sqlalchemy.engine.Engine COMMIT


In this example, the two SQL statements that are generally `transactional`, a `CREATE TABLE statement` and an `INSERT statement` that's parameterized. As we want the work we've done to be committed (saved) within our block, we invoke the `Connection.commit()` method which commits the transaction. After we call this method inside the block, we can continue to run more SQL statements and if we choose we may call `Connection.commit()` again for subsequent statements. SQLAlchemy refers to this style as **commit as you go**.

There is also another style of committing data, which is that we can declare our `connect` block to be a transaction block up front. For this mode of operation, we use the `Engine.begin()` method to acquire the connection, rather than the `Engine.connect()` method. This method will both manage the scope of the `Connection` and also enclose everything inside of a transaction with `COMMIT at the end, assuming a successful block`, or `ROLLBACK in case of exception raise`. This style may be referred towards as **begin once**.

In [4]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO table_name (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )

2022-09-19 06:22:52,385 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-19 06:22:52,387 INFO sqlalchemy.engine.Engine INSERT INTO table_name (x, y) VALUES (?, ?)
2022-09-19 06:22:52,389 INFO sqlalchemy.engine.Engine [cached since 35.89s ago] ((6, 8), (9, 10))
2022-09-19 06:22:52,390 INFO sqlalchemy.engine.Engine COMMIT


> `Begin once` style is often preferred as it is more succinct and indicates the intention of the entire block up front.

> ##### What’s `BEGIN (implicit)`?
> You might have noticed the log line `BEGIN (implicit)` at the start of a transaction block. `implicit` here means that SQLAlchemy did not actually send any command to the database; it just considers this to be the start of the DBAPI's implicit transaction. You can register event hooks to intercept this event, for example.